In [ ]:
# Final as of now good
import cv2
import numpy as np

# Path to the input video file (update as needed)
video_path = '/content/drive/MyDrive/Avawatz/internship documentation/D28_20231018123011.mp4'

# Path to the output video file (update as needed)
output_path = 'finalized.mp4'

# Create a VideoCapture object to read the input video
cap = cv2.VideoCapture(video_path)

# Create a background subtractor using the GMM model (MOG2)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=False)

# Get video frame properties (width, height, frames per second)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Calculate the number of frames for the first 30 seconds
frame_limit = int(fps * 30)  # fps * 30 seconds

# Frame counter to track the number of frames processed
frame_counter = 0

# Create a structuring element for morphological operations
kernel = np.ones((3, 3), np.uint8)

# Main loop to read frames and perform background subtraction
while cap.isOpened() and frame_counter < frame_limit:
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no frame is retrieved (end of video)

    # Apply background subtraction to the frame
    fg_mask = background_subtractor.apply(frame)

    # Convert the mask to binary (thresholding) for better results
    _, fg_mask = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY)

    # Apply morphological erosion to the binary mask to reduce noise
    fg_mask = cv2.erode(fg_mask, kernel, iterations=2)  # Apply erosion twice

    # Apply morphological dilation to the binary mask to smooth the mask
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=7)  # Apply dilation

    # Use the mask to extract the foreground from the original frame
    foreground = cv2.bitwise_and(frame, frame, mask=fg_mask)

    # Add text based on motion detection
    if np.sum(fg_mask) > 0:
        text = "Motion"
    else:
        text = "No Motion"
    cv2.putText(foreground, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Save the foreground frame to the output video
    output_video.write(foreground)

    # Increment the frame counter
    frame_counter += 1

# Release the VideoCapture and VideoWriter objects
cap.release()
output_video.release()

# Output message indicating the saved output video file
print(f'Output video saved at: {output_path}')

Output video saved at: finalized.mp4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
